### Tobalaba ENS playground with web3.py
With FIFS registrar and .ewf as TLD 

In [ ]:
import os
import json

from web3 import Web3

# we cannot use the namehash function from the ENS API because it doesn't support .ewf TLDs
from ensutils import name_to_hash, label_to_hash

# ENS contract addresses
registry_address = Web3.toChecksumAddress("0x76b4d318cf896288fd44315e93c891ad4017aa14")
registrar_address = Web3.toChecksumAddress("0xf106c25e7624726dfa52a25f4fef29ec6a8ce103")
resolver_address = Web3.toChecksumAddress("0x1da4c76ed136f32693700714502c6dacd62f5a3c")
reverse_resolver_address = Web3.toChecksumAddress("0x9d17a1ef45df65af5cdbfc98e3bfe3e90463780e")
reverse_registrar_address = Web3.toChecksumAddress("0x89a3ae0d90304b00cea94c0f5528b1c657b041cc")

# location of contract abis
# change this if needed
contractfolder = os.path.join(os.getcwd(), "abis")

# rpc address
http_rpc = "http://127.0.0.1:8545"

#reading abis
with open(os.path.join(contractfolder, 'ENSRegistry.abi')) as f:
    registry_abi = json.load(f)
    
with open(os.path.join(contractfolder, 'FIFSRegistrar.abi')) as f:
    registrar_abi = json.load(f)
    
with open(os.path.join(contractfolder, 'PublicResolver.abi')) as f:
    resolver_abi = json.load(f)

with open(os.path.join(contractfolder, 'ResolverInterface.abi')) as f:
    resolver_interface_abi = json.load(f)

with open(os.path.join(contractfolder, 'DefaultReverseResolver.abi')) as f:
    reverse_resolver_abi = json.load(f)

with open(os.path.join(contractfolder, 'ReverseRegistrar.abi')) as f:
    reverse_registrar_abi = json.load(f)

In [ ]:
# setting provider
w3 = Web3(Web3.HTTPProvider(http_rpc))

# you have to make the ENS API point to the 
# ENS-Registry contract address if you want to use the ENS API
ns = w3.ens.fromWeb3(web3=w3, addr=registry_address)

#set the default account to use and unlock it
w3.eth.defaultAccount = w3.eth.accounts[0]
w3.personal.unlockAccount(w3.eth.defaultAccount, "yourpassword")

In [ ]:
# Instantiate the deployed registry and resolver contracts to use
registry = w3.eth.contract(address=registry_address, abi=registry_abi)
registrar = w3.eth.contract(address=registrar_address, abi=registrar_abi)
resolver = w3.eth.contract(address=resolver_address, abi=resolver_abi)
reverse_resolver = w3.eth.contract(address=reverse_resolver_address, abi=reverse_resolver_abi)
reverse_registrar = w3.eth.contract(address=reverse_registrar_address, abi=reverse_registrar_abi)

## Doing lookups and reverse lookups

### Resolving names (lookup)
 1. Get resolver address of a node from registry
 2. Instantiate resolver contrac with resolver interface abi
 2. Fetch registered address (or whatever you need)

In [ ]:
abi_of_resolver = resolver_abi
reg_addr = registry.functions.resolver(name_to_hash("nametocheck.ewf")).call()

# check if resolver is actually set
if reg_addr != "0x" + "0" * 40:
    res = w3.eth.contract(address=Web3.toChecksumAddress(reg_addr), abi=resolver_interface_abi)

    print(res.functions.addr(name_to_hash("nametocheck.ewf")).call())

### Resolving addresses (reverse lookup)
 1. Get resolver address of the address node from registry
 2. Instantiate reverse resolver contract
 2. Fetch canonical name for the address

In [ ]:
# More explanation below at "Step 4 - Set reverse lookup"
address_of_interest = w3.eth.accounts[0]
address_node_hash = reverse_registrar.functions.node(address_of_interest).call()
custom_reverse_resolver = w3.eth.contract(address=Web3.toChecksumAddress(reverse_resolver_address), abi=reverse_resolver_abi)
registered_name = custom_reverse_resolver.functions.name(address_node_hash).call()

print(registered_name)

## Registering and managing names (nodes)

### Step 1 - Check if a name is available to register

In [ ]:
# check ownership directly interacting with the registry contract
ADDRESS_EMPTY = "0x" + "0" * 40 
print(registry.functions.owner(name_to_hash("mycompany.ewf")).call() == ADDRESS_EMPTY)

### Step 2 - Registering ownership of a name
Claim ownership of your desired domain (FIFS registrar: first come, first served)
It will be registered under the .ewf top level domain

In [ ]:
nameowner = w3.eth.accounts[0]
txhash = registrar.functions.register(label_to_hash("mycompany"), nameowner).transact()
txreceipt = w3.eth.waitForTransactionReceipt(txhash)

In [ ]:
# change owner of a node
newowner = Web3.toChecksumAddress("new_owner_address_here")
registry.functions.setOwner(label_to_hash("mycompany"), newowner).transact()
txreceipt = w3.eth.waitForTransactionReceipt(txhash)

### Step 3 - Mapping an address to your name
Map the name to point to your desired address in a Resolver contract
 - Feel free to use the supplied public resolver, but you can also deploy your own
 - You need to have ownership of the name already
 - Give it a lil gas

In [ ]:
point_to_address = Web3.toChecksumAddress(w3.eth.accounts[0])
txhash = resolver.functions.setAddr(name_to_hash("mycompany.ewf"), point_to_address).transact({"gas": 8000000})
txreceipt = w3.eth.waitForTransactionReceipt(txhash)

In [ ]:
# verify the result - wait for like 5 secs till it get mined
assert resolver.functions.addr(name_to_hash("mycompany.ewf")).call() == point_to_address

In [ ]:
# change resolver of a node
new_resolver_address = Web3.toChecksumAddress("your_resolver_address_here")
txhash = registry.functions.setResolver(name_to_hash("mycompany.ewf"), new_resolver_address).transact()
txreceipt = w3.eth.waitForTransactionReceipt(txhash)

### Step 4 -  Set reverse lookup
Now you also want others to check the canonical name that belongs to your address. Multiple names can belong to an address, but you can set only one for reverse lookups. This step is optional.

Reverse Registrar holds the ".addr.reverse" top domain.
Address nodes are registered in the form of "< hex account address >.addr.reverse".
To reverse lookup the name for an address, just simply do a lookup to < hex account address >.addr.reverse. Don't forget to remove the the leading '0x' from the address.


In [ ]:
# Registering the reverse record

# Calling account must be the owner of the name
nameowner = w3.eth.accounts[0]
txhash = reverse_registrar.functions.setName("mycompany.ewf").transact({"from": nameowner})
txreceipt = w3.eth.waitForTransactionReceipt(txhash)

In [ ]:
# Doing a reverse lookup method 1 - getting the address node hash from the registrar contract
addressnode_namehash1 = reverse_registrar.functions.node(nameowner).call()
registered_name_1 = reverse_resolver.functions.name(addressnode_namehash1).call()

# Doing a reverse lookup method 2 - caculating the addres node hash yourself
# you have to cut down the '0x' beginning of the address string
addressnode_namehash2 = name_to_hash(str(nameowner)[2:]+".addr.reverse")
registered_name_2 = reverse_resolver.functions.name(addressnode_namehash2).call()

# Doing a reverse lookup method 3 - if a custom/different resolver was used then the provided one,
# first you can get its address from the registry
custom_reverse_resolver_addr = registry.functions.resolver(addressnode_namehash1).call()
# instantiate using its abi, then proceed with a regular lookup as in version 1 or 2
# in this case it is the same abi as before (this is just a showcase)
custom_reverse_resolver = w3.eth.contract(address=Web3.toChecksumAddress(custom_reverse_resolver_addr), abi=reverse_resolver_abi)
registered_name_3 = custom_reverse_resolver.functions.name(addressnode_namehash1).call()

assert registered_name_1 == registered_name_2 == registered_name_3

print("{} reverse lookup: {}".format(nameowner, registered_name_1))

### Step 5 - Register subnodes
Once you have ownership of a name (node), you have the ability to register sub-nodes as well.
E.g: adam.yourdomain.ewf
You have to be the owner of the root domain (in this case yourdomain.ewf) in order to register a subnode.

In [ ]:
# I want to register my name (adam) as a subdomain under mycompany.ewf => adam.mycompany.ewf
# in this case I register the subnode for my account as well
subnodeowner = w3.eth.accounts[0]
txhash = registry.functions.setSubnodeOwner(name_to_hash("mycompany.ewf"), label_to_hash("adam"), subnodeowner).transact({"gas": 8000000})
txreceipt = w3.eth.waitForTransactionReceipt(txhash)

In [ ]:
# verify the ownership transfer - wait for like 5 secs till it get mined
assert registry.functions.owner(name_to_hash("adam.mycompany.ewf")).call() == subnodeowner